### Libraries

In [1]:
import sys
sys.path.insert(0,'/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

### Script settings

In [2]:
close_workbook = 'yes'

csv_path = '/Users/jarad/Fake Folder/Accounts and Biz Dev/Recurring/Reseller Discount Percentage Check/CSVs/'
workbook_title = '2019 - 02 - Feb - Reseller Discount Percent Check'

### Get cost data

In [3]:
msrp = pd.read_sql(
'''
SELECT
p.part_id,
pd.products_name AS part_name,
k.sku_id,
IF(k.sku_outsourced_assembly = 1, pm.consigned_latest_cost, pm.latest_cost) AS latest_cost,
IF(k.sku_outsourced_assembly = 1, pm.consigned_manual_cost, pm.manual_cost) AS manual_cost,
p.products_price
FROM parts p
JOIN skus k ON p.part_id = k.part_id
AND k.sku_status = 'working'
JOIN products_manufacturing pm ON k.sku_id = pm.sku_id
JOIN products_description pd ON p.part_id = pd.part_id
''', db)

col_fix(msrp)
print('%s part_id dupe(s)' % np.sum(msrp['part id'].duplicated()))

53 part_id dupe(s)


### Get discount data

In [4]:
dis1 = pd.read_sql(
'''
SELECT
part_id,
discount_price,
'products_reseller_discount_pricing' AS 'table'
FROM products_reseller_discount_pricing
''', db)

dis2 = pd.read_sql(
'''
SELECT
part_id,
discount_price,
'premier_discount_pricing' AS 'table'
FROM premier_discount_pricing
''', db)

dis3 = pd.read_sql(
'''
SELECT
part_id,
discount_price,
'products_discount_quantity' As 'table'
FROM products_discount_quantity
''', db)

dis_main = pd.concat([dis1,dis2,dis3])
dis_main.reset_index(drop = True, inplace = True)

col_fix(dis_main)
dis_main.rename(columns = {'discount price':'percentage off'}, inplace = True)
dis_main['percentage off'] = dis_main['percentage off']/100
dis_main.drop_duplicates(['part id','percentage off'], inplace = True)

### Structure it 

In [5]:
# merge
msrp_check = pd.merge(dis_main, msrp, how = 'right', on = 'part id')

# remove parts for which we do not offer discounts of any kind
msrp_check.dropna(inplace = True)

# get price after discount
msrp_check['products price after discount'] = msrp_check['products price'] * (1 - msrp_check['percentage off'])

# get gross profits
for x in ['latest cost','manual cost']:
    msrp_check[x + ' profit'] = msrp_check['products price after discount'] - msrp_check[x]

# calculate this    
msrp_check['margin'] = msrp_check[['latest cost profit','manual cost profit']].min(axis = 1)/msrp_check['products price']        
msrp_check['margin'] = np.round(msrp_check['margin'], 3)

msrp_check['margin @ discount price'] = msrp_check[['latest cost profit','manual cost profit']].min(axis = 1)/msrp_check['products price after discount']        
msrp_check['margin @ discount price'] = np.round(msrp_check['margin @ discount price'], 3)

# get any line < 0 or any % greater than 30%
old_cols = msrp_check.columns

new_cols = ['part id',
        'sku id',
        'part name',
        'latest cost',
        'manual cost',
        'products price',
        'percentage off',
        'products price after discount',
        'latest cost profit',
        'manual cost profit',
        'margin',
        'margin @ discount price',
        'table']
# check to make sure you got all columns
if len(old_cols) == len(new_cols):
    msrp_check = msrp_check[new_cols]
    pass
else:
    raise ValueError('you left out these columns:')
    print(set(list(old_cols.columns)) - set(new_cols))
    
msrp_check.sort_values('margin', inplace = True)    

### Where discount > 30%

In [10]:
greater = msrp_check[msrp_check['percentage off'] > 0.30]

a = list(set(greater['table']))

ls = []
for i in a:
    if i != 'premier_discount_pricing':
        ls.append(i)
        
if len(ls) == 0:
    print('no applicable table has discounts over 30%')
else:
    display(greater[greater['table'].isin(ls)])

no applicable table has discounts over 30%


### Where there is a profit loss or no profit at some discount %

In [7]:
loss = msrp_check[(msrp_check['margin'] <= 0) | (msrp_check['margin @ discount price'] <= 0)]

if loss.empty:
    print('no losses')
else:
    display(loss)

no losses


### Write to Excel

In [8]:
if close_workbook == 'yes':    
    writer = pd.ExcelWriter(csv_path + workbook_title + '.xlsx')
    
    msrp_check.to_excel(writer, 'all data', index = False)
    greater.to_excel(writer, 'greater than 30%', index = False)
    loss.to_excel(writer, 'margin is zero or negative', index = False)
    
    writer.save()

In [9]:
print('done')

done
